# Прогнозирование оттока пользователей провайдера телекоммуникационных услуг

В нашем распоряжении находятся данные о клиентах провайдера. Необходимо спрогнозировать - уйдет клиент или нет.

**Описание данных:**

Таблица train содержит данные пользователей. Где:
- 1 - клиент ушел,
- 0 - остался.

Таблица log — содержит данные обращения пользователей

Таблица named — лог днс-запросов к доменам конкурентов (rt.ru и sampo.ru).

Таблица type_contract - тип списания у пользователей, где:
- 1 - посуточная,
- 0 - помесячная.

## Подготовка данных

In [ ]:
!pip install catboost -q
!pip install optuna -q

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
import catboost
import lightgbm as lgb
import optuna
from numpy import argmax
from numpy import arange
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_randfloat
from sklearn.metrics import make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_train = pd.read_csv("/content/train_dataset_train.csv")
df_sample_solution = pd.read_csv("/content/sample_solution.csv")

df_named = pd.read_csv("/content/named.csv")
df_type_contract = pd.read_csv("/content/type_contract.csv")
df_log = pd.read_csv("/content/log.csv")

In [ ]:
df_train.head()

,contract_id,blocked
0,7780,0
1,7785,0
2,7794,0
3,7795,0
4,7798,0


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   contract_id  5992 non-null   int64
 1   blocked      5992 non-null   int64
dtypes: int64(2)
memory usage: 93.8 KB


In [ ]:
df_type_contract.head()

,contract_id,day_or_month_contract
0,7780,0
1,3996,0
2,7785,0
3,7794,0
4,7795,1


In [ ]:
df_type_contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6805 entries, 0 to 6804
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   contract_id            6805 non-null   int64
 1   day_or_month_contract  6805 non-null   int64
dtypes: int64(2)
memory usage: 106.5 KB


In [ ]:
df_named.head()

,date,url,contract_id
0,17-Apr-2021,webmail.sampo.ru,101397.0
1,17-Apr-2021,webmail.sampo.ru,179624.0
2,17-Apr-2021,tvip-provision.sampo.ru,190335.0
3,17-Apr-2021,tvip-provision.sampo.ru,61670.0
4,17-Apr-2021,tvip-provision.sampo.ru,39370.0


In [ ]:
df_named.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5897793 entries, 0 to 5897792
Data columns (total 3 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         object 
 1   url          object 
 2   contract_id  float64
dtypes: float64(1), object(2)
memory usage: 135.0+ MB


In [ ]:
df_named.isna().sum()

date           0
url            1
contract_id    1
dtype: int64

In [ ]:
df_named = df_named.dropna()

Класетризируем все обращения к конкурентам по наименованию компании.

In [ ]:
def url_category(url):
    if 'sampo.ru' in url:
        return 'sampo'
    return 'rt'
df_named['competitor'] = df_named['url'].apply(url_category)

In [ ]:
df_named

,date,url,contract_id,competitor
0,17-Apr-2021,webmail.sampo.ru,101397,sampo
1,17-Apr-2021,webmail.sampo.ru,179624,sampo
2,17-Apr-2021,tvip-provision.sampo.ru,190335,sampo
3,17-Apr-2021,tvip-provision.sampo.ru,61670,sampo
4,17-Apr-2021,tvip-provision.sampo.ru,39370,sampo
...,...,...,...,...
5897787,22-Apr-2021,camera.rt.ru,142736,rt
5897788,22-Apr-2021,camera.rt.ru,195218,rt
5897789,22-Apr-2021,camera.rt.ru,114658,rt
5897790,22-Apr-2021,fnc.rt.ru,133211,rt


In [ ]:
df_named['contract_id'] = df_named['contract_id'].astype('int')


In [ ]:
df_named.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5897792 entries, 0 to 5897791
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   date         object
 1   url          object
 2   contract_id  int64 
 3   competitor   object
dtypes: int64(1), object(3)
memory usage: 225.0+ MB


In [ ]:
df_log

,contract_id,event_date,event_type
0,36294,2021-03-28 16:24:30,Добавление в Обращались с номеров
1,36294,2021-03-28 16:27:41,Обращение в службу заботы о клиентах
2,36294,2021-03-28 16:29:56,Выключение IPTV-пакета
3,36294,2021-03-28 16:29:56,Включение IPTV-пакета
4,36294,2021-03-28 16:35:04,Обращение в службу заботы о клиентах
...,...,...,...
25269,11801,2021-04-29 07:50:44,Информер ВК. Показ
25270,54405,2021-03-17 10:38:42,Информер ВК. Показ
25271,54405,2021-03-23 08:21:29,Информер ВК. Показ
25272,54405,2021-04-09 13:04:26,Информер ВК. Показ


Объединим таблицы.

In [ ]:
df = pd.merge(df_train, df_type_contract , on="contract_id", how='left').drop_duplicates(subset="contract_id")
df = pd.merge(df, df_named.groupby('contract_id')['url'].count() , on="contract_id", how='left')
df = pd.merge(df, df_log.groupby('contract_id')['event_date'].count(), on="contract_id", how='left')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 5991
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   contract_id            5992 non-null   int64  
 1   blocked                5992 non-null   int64  
 2   day_or_month_contract  5990 non-null   float64
 3   url                    2559 non-null   float64
 4   event_date             3809 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 280.9 KB


In [ ]:
df = df.fillna(0)

In [ ]:
df

,contract_id,blocked,day_or_month_contract,url,event_date
0,7780,0,0.0,83.0,9.0
1,7785,0,0.0,8.0,1.0
2,7794,0,0.0,0.0,0.0
3,7795,0,1.0,0.0,7.0
4,7798,0,0.0,0.0,3.0
...,...,...,...,...,...
5987,274601,1,1.0,0.0,6.0
5988,274710,1,0.0,0.0,7.0
5989,274782,1,0.0,0.0,8.0
5990,274786,1,1.0,1.0,9.0


Разделим на признаки и целевой признак.

In [ ]:
X = df2.drop(['contract_id', 'blocked'], axis = 1)
y = df2['blocked']

In [ ]:
y.value_counts(normalize=True)

0    0.883511
1    0.116489
Name: blocked, dtype: float64

Целевой признак сильно разбалансирован.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)


Сбалансируем классы.

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(X_train, y_train, 8)

Создадим scorer.

In [ ]:
scorer=make_scorer(recall_score, average='macro')

Создадим функцию для добавления результатов работы моделей.

In [ ]:
results = {'name': [], 'best_params':[], 'Recall':[]}

def add_model_result(results, name, best_params, recall):
    results['name'].append(name)
    results['best_params'].append(best_params)
    results['Recall'].append(recall)

## Обучение моделей

### LogisticRegression

In [ ]:
preprocessing = ColumnTransformer(
    [
        (
            "object",
            MinMaxScaler(),
            ["competitor",	"event_date"],
        )
   ]
)

pipeline = Pipeline(
    [
        ("prep", preprocessing),
        ("clf", LogisticRegression()),
    ]
)
parameters = {"clf__C":[1, 10, 100, 1000]}
grid_lr = GridSearchCV(pipeline, parameters, scoring=scorer, cv=3)
grid_lr.fit(features_upsampled, target_upsampled)

In [ ]:
print('LogisticRegression', grid_lr.best_score_, grid_lr.best_params_)

LogisticRegression 0.5280051729769647 {'clf__C': 1000}


In [ ]:
model = LogisticRegression()

parameters = [#{'penalty':['l1','l2']}, 
              {'C':[0.01, 0.1, 1, 2, 10, 100, 1000]}]

grid_lr = GridSearchCV(model, parameters, scoring=scorer, cv=5)
grid_lr.fit(features_upsampled, target_upsampled)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [0.01, 0.1, 1, 2, 10, 100, 1000]}],
             scoring=make_scorer(recall_score, average=macro))

In [ ]:
print('LogisticRegression', grid_lr.best_score_, grid_lr.best_params_)

LogisticRegression 0.5856266554426187 {'C': 1}


In [ ]:
add_model_result(results, 'LogisticRegression', grid_lr.best_params_, grid_lr.best_score_)

### LightGBM 

In [ ]:
model = lgb.LGBMClassifier()
recall = (cross_val_score(model,features_upsampled, target_upsampled, scoring=scorer, cv=3)).mean()
print('recall = ', recall)

recall =  0.680994397492004


In [ ]:
add_model_result(results, 'LightGBM', 'None',recall)

In [ ]:
model = lgb.LGBMClassifier()
model.fit(features_upsampled, target_upsampled)
predict = model.predict_proba(X_test)[:, 1]
thresholds_lgb = arange(0, 1, 0.001)
scores = [recall_score(y_test, to_labels(predict, t), average='macro') for t in thresholds_lgb]
ix = argmax(scores)
print('Threshold=%.3f, Recall=%.5f' % (thresholds_lgb[ix], scores[ix]))

Threshold=0.445, Recall=0.61222


In [ ]:
preprocessing = ColumnTransformer(
    [
        (
            "object",
            OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
            ["competitor",	"event_type"],
        )
   ]
)

pipeline = Pipeline(
    [
        ("prep", preprocessing),
        ("clf", lgb.LGBMClassifier()),
    ]
)
parameters = {'clf__num_leaves' : [20,40,60,80,100],
              'clf__max_depth' :range(1, 15, 3),
              'clf__reg_alpha':[0,0.01,0.03]}

grid_lgb_pip = GridSearchCV(pipeline, parameters, scoring=scorer, cv=5)
grid_lgb_pip.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('object',
                                                                         OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                        unknown_value=-1),
                                                                         ['competitor',
                                                                          'event_type'])])),
                                       ('clf', LGBMClassifier())]),
             param_grid={'clf__max_depth': range(1, 15, 3),
                         'clf__num_leaves': [20, 40, 60, 80, 100],
                         'clf__reg_alpha': [0, 0.01, 0.03]},
             scoring=make_scorer(recall_score, average=macro))

In [ ]:
print('LightGBM', grid_lgb_pip.best_score_, grid_lgb_pip.best_params_)

LightGBM 0.5287986436259511 {'clf__max_depth': 13, 'clf__num_leaves': 40, 'clf__reg_alpha': 0.01}


In [ ]:
%%time
model = lgb.LGBMClassifier()
parameters = {'num_leaves' : [20,40,60,80,100],
              'max_depth' : sp_randint(1, 10),
              'reg_alpha':[0,0.01,0.03]}

grid_lgb = RandomizedSearchCV(model,parameters, scoring=scorer, cv=5)
grid_lgb.fit(features_upsampled, target_upsampled)

CPU times: user 9 s, sys: 302 ms, total: 9.31 s
Wall time: 5.98 s


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(),
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9f7d4ab050>,
                                        'num_leaves': [20, 40, 60, 80, 100],
                                        'reg_alpha': [0, 0.01, 0.03]},
                   scoring=make_scorer(recall_score, average=macro))

In [ ]:
print('LightGBM', grid_lgb.best_score_, grid_lgb.best_params_)

LightGBM 0.6770978461623366 {'max_depth': 7, 'num_leaves': 20, 'reg_alpha': 0.03}


### CatBoost

In [ ]:
%%time
model = CatBoostClassifier(silent=True)

parameters = {'depth' : sp_randint(4, 10),
              'learning_rate': sp_randfloat(0.01, 0.06),
              'iterations' : sp_randint(500, 1000)}

grid_cat = RandomizedSearchCV(model,parameters, scoring=scorer, cv=3)
grid_cat.fit(features_upsampled, target_upsampled)

CPU times: user 1min 33s, sys: 13.8 s, total: 1min 47s
Wall time: 1min 3s


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7f9f7d4d4890>,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9f7d560fd0>,
                                        'iterations': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9f7c7cc4d0>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9f7c7cc950>},
                   scoring=make_scorer(recall_score, average=macro))

In [ ]:
print('CatBoost', grid_cat.best_score_, grid_cat.best_params_)

CatBoost 0.7015791111878982 {'depth': 9, 'iterations': 969, 'learning_rate': 0.06457843179086609}


In [ ]:
add_model_result(results, 'CatBoost', grid_cat.best_params_, grid_cat.best_score_)

### Результаты

In [ ]:
pd.DataFrame(results).sort_values('Recall', ascending=False)

,name,best_params,Recall
2,CatBoost,"{'depth': 9, 'iterations': 969, 'learning_rate...",0.701579
1,LightGBM,None,0.680994
0,LogisticRegression,{'C': 1},0.585627


## Сохранение результатов

In [ ]:
df_solution = pd.merge(df_sample_solution, df_type_contract , on="contract_id", how='left').drop_duplicates(subset="contract_id")
df_solution = pd.merge(df_solution, df_named.groupby('contract_id')['url'].count() , on="contract_id", how='left')
df_solution = pd.merge(df_solution, df_log.groupby('contract_id')['event_date'].count(), on="contract_id", how='left')

In [ ]:
df_solution

,contract_id,blocked,day_or_month_contract,url,event_date
0,3453,0,0,NaN,1.0
1,3454,0,1,NaN,1.0
2,3455,0,1,24.0,NaN
3,3456,0,0,NaN,4.0
4,3457,0,0,NaN,4.0
...,...,...,...,...,...
805,4258,0,1,NaN,3.0
806,4259,0,1,478.0,NaN
807,4260,0,0,9.0,3.0
808,4261,0,0,NaN,18.0


In [ ]:
df_solution = df2_solution.fillna(0)

In [ ]:
X_train = pd.concat([X_train, X_test])
y_train = pd.concat([y_train, y_test])
features_upsampled, target_upsampled = upsample(X_train, y_train, 8)

In [ ]:
X_test = df_solution.drop(['contract_id', 'blocked'], axis = 1)

In [ ]:
model = CatBoostClassifier(depth = 9, iterations = 969, learning_rate = 0.06457843179086609, silent=True)
model.fit(features_upsampled, target_upsampled)
df2_solution['blocked'] = model.predict(X_test) 

In [ ]:
df_solution = df_solution[['contract_id', 'blocked']]

In [ ]:
df_solution

,contract_id,blocked
0,3453,1
1,3454,1
2,3455,1
3,3456,1
4,3457,1
...,...,...
805,4258,1
806,4259,0
807,4260,0
808,4261,1


In [ ]:
df_solution.to_csv('solution.csv', index=False) 

## Вывод

Подготовлены данные и построена модель прогнозирование оттока пользователей провайдера телекоммуникационных услуг. Лучшей моделью машинного обучения стала CatBoost  с параметрами: `depth = 9, iterations = 969, learning_rate = 0.06457843179086609`.